### This Juypter notebook contains code improvisions to the summarization and introduces a method for plagarism check by similarity check
1. Model Selection
2. Plagarism checker


In [1]:
!pip install transformers
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=50db9919b7ff863d70ef347ea716299cfd73344d87e8c4058e924fb65b9af04e
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [2]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summarizer_2 = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

Replace ARTICLE variable with the text you want to summarize

In [3]:
ARTICLE = " A multi-model ensemble is constructed with the global climate models that are found to best reproduce the wind climate in Northern Europe. The results anticipate an overall decline in wind power density, especially in the high-emissions scenario and in certain regions (up to 30% off Western Ireland), which should be taken into account in planning future offshore wind deployments. As an exception,slight increases (around 10%) are projected in certain areas of the Baltic Sea. The general decline is less pro-nounced in the low-emissions scenario. Indeed, the results prove that reducing emissions as advocated by current climate objectives would not only weaken the declining trend but also lead to a more stable resource. 1. Introduction Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world. As a result, investments in the energy sector are shifting towards renewable energies; among all the renewables, wind energy is experiencing remarkable growth in recent years. Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 [1] and 94 GW in 2021 [2]."

In [4]:
# Gives the words and character count of the input text
print(len(ARTICLE.split()))
print(len(ARTICLE))

197
1245


#### Hyperparameters used here are 
1. Max_length: Determines the MAXIMUM number of output words generated
2. Min_length: Determines the MINIMUM number of output words generated

In [5]:
summarized_text = summarizer(ARTICLE, max_length=190, min_length=170, do_sample=False)

In [6]:
summarized_text_2 = summarizer_2(ARTICLE, max_length=190, min_length=170, do_sample=False)

In [7]:
print(summarized_text)

[{'summary_text': 'A multi-model ensemble is constructed with the global climate models that are found to best reproduce the wind climate in Northern Europe. The results anticipate an overall decline in wind power density, especially in the high-emissions scenario and in certain regions. Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 and 94 GW in 2021 [2]. The results prove that reducing emissions as advocated by current climate objectives would not only weaken the declining trend but also lead to a more stable resource, the authors say. The findings should be taken into account in planning future offshore wind deployments, they add. The authors conclude that wind energy is experiencing remarkable growth in recent years and should be considered as a source of energy for the future of Northern Europe and the rest of the world. They conclude that the results should be used to inform future planning of offshore wind

In [8]:
print(summarized_text_2)

[{'summary_text': ' A multi-model ensemble is constructed with the global climate models that are found to best reproduce the wind climate in Northern Europe . The results anticipate an overall decline in wind power density, especially in the high-emissions scenario and in certain regions (up to 30% off Western Ireland), which should be taken into account in planning future offshore wind deployments . Globally, the wind energy sector has undergone two record years in new capacity installations – a total of 93 GW in 2020 [1] and 94 GW in 2021 [2] Globally. Globally,. the wind . energy sector . has undergone . two record year in new . capacity installations - a total . of 93GW in 2020 and 94GW in 2021, respectively [2]. The results prove that reducing emissions would not only weaken the declining trend but also lead to a more stable resource'}]


#### Cosine similarity is used here to measure similarity between generated summarized texts and and the main text(ARTICLE)

In [9]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
print(ARTICLE)
print(summarized_text[0]["summary_text"])
print(summarized_text_2[0]["summary_text"])

 A multi-model ensemble is constructed with the global climate models that are found to best reproduce the wind climate in Northern Europe. The results anticipate an overall decline in wind power density, especially in the high-emissions scenario and in certain regions (up to 30% off Western Ireland), which should be taken into account in planning future offshore wind deployments. As an exception,slight increases (around 10%) are projected in certain areas of the Baltic Sea. The general decline is less pro-nounced in the low-emissions scenario. Indeed, the results prove that reducing emissions as advocated by current climate objectives would not only weaken the declining trend but also lead to a more stable resource. 1. Introduction Cutting carbon emissions to mitigate climate change and reducing the over-dependence on fossil fuels are listed as a priority in a great number of countries throughout the world. As a result, investments in the energy sector are shifting towards renewable e

In [11]:
from sentence_transformers import SentenceTransformer
model_name = "sentence-transformers/all-MiniLM-L6-v2"
bert_model = SentenceTransformer(model_name)

In [12]:
ARTICLE_enc = bert_model.encode(ARTICLE)
summarized_text_enc = bert_model.encode(summarized_text)
summarized_text_enc_2 = bert_model.encode(summarized_text_2)

In [13]:
sim=cosine_similarity(ARTICLE_enc.reshape(1, -1),summarized_text_enc.reshape(1, -1))[0][0]
print(sim)

0.92252946


In [16]:
sim_2=cosine_similarity(ARTICLE_enc.reshape(1, -1),summarized_text_enc_2.reshape(1, -1))[0][0]
print(sim_2)

0.8918052


In [17]:
sim_3=cosine_similarity(summarized_text_enc.reshape(1, -1),summarized_text_enc_2.reshape(1, -1))[0][0]
print(sim_3)

0.9066205
